### Essential files for Azure Storage

In [0]:
pip install azure-storage-blob==2.1.0

Python interpreter will be restarted.
Requirement already satisfied: azure-storage-blob==2.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5bc14e77-a3ec-4083-886b-a7d9097be902/lib/python3.8/site-packages (2.1.0)
Requirement already satisfied: azure-common>=1.1.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5bc14e77-a3ec-4083-886b-a7d9097be902/lib/python3.8/site-packages (from azure-storage-blob==2.1.0) (1.1.27)
Requirement already satisfied: azure-storage-common~=2.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-5bc14e77-a3ec-4083-886b-a7d9097be902/lib/python3.8/site-packages (from azure-storage-blob==2.1.0) (2.1.0)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.8/site-packages (from azure-storage-common~=2.1->azure-storage-blob==2.1.0) (2.8.1)
Requirement already satisfied: cryptography in /databricks/python3/lib/python3.8/site-packages (from azure-storage-common~=2.1->azure-storage-blob==2.1.0) (3.1.1)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from azure-storage-common~=2.1->azure-storage-blob==2.1.0) (2.24.0)
Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.8/site-packages (from python-dateutil->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (1.15.0)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.8/site-packages (from cryptography->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (1.14.3)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (2020.12.5)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (1.25.11)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi!=1.11.3,>=1.8->cryptography->azure-storage-common~=2.1->azure-storage-blob==2.1.0) (2.20)
WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5bc14e77-a3ec-4083-886b-a7d9097be902/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from azure.storage.blob import AppendBlobService

In [0]:
import os, uuid, sys


In [0]:
import numpy as np
import urllib
import random
import json
from sklearn.model_selection import train_test_split
import pathlib
from datetime import datetime, timedelta
import requests
import pandas as pd
import datetime
from datetime import datetime

### Connect to Storage account

### Function to fetch

In [0]:
def fetch_data_db(username,password):
        global values
        global results
        ##following is example of unauthenticated read... should fail if the db is set to not open read
        url = "http://gapp.agverse.in/api/v1/get-telemetry-data/"
        #response = urllib.request.urlopen(url)
        #this is for authenticated reading
        url_auth=urllib.request.Request(url,headers={"username" : username ,"password" : password})
        response = urllib.request.urlopen(url_auth)
        data = response.read()
        values = json.loads(data)
        results = pd.DataFrame(values)
        return results

[Refer this document for complete steps](https://docs.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-directory-file-acl-python)

### Function to create a dataframe of data obtained from api link

In [0]:
def restAPI_data_to_dataFrame(username,password):
    id = []
    user_id = []
    animal_id = []
    telemetry_data = []
    mac_id = []
#     global timestamp 
    timestamp = []
    results = fetch_data_db(username,password)
    x = results.result 

    for i in x:
        id.append(i['id'])
        user_id.append(i['user_id'])
        animal_id.append(i['animal_id'])
        telemetry_data.append(i['telemetry_data'])
        mac_id.append(i['mac_id'])
        timestamp.append(i['timestamp'])
    _dict_ = {'id' : id,'user_id' : user_id,'animal_id' : animal_id,'telemetry_data' : telemetry_data,'mac_id' : mac_id,'timestamp':timestamp} 
    Data = pd.DataFrame(_dict_)
    return Data

### Function which will transform csv data to dictionary

In [0]:
def csv_to_dict(username,password):
  values = fetch_data_db(username,password)
  data_format_dict = {}
  for animal_data in values['result']:
    cow_data = {}
    cow_data['user_id'] = animal_data['user_id']
    cow_data['telemetry_data'] = animal_data['telemetry_data']
    cow_data['mac_id'] = animal_data['mac_id']
    cow_data['timestamp'] = animal_data['timestamp']
    animal_id = animal_data['animal_id']
    if animal_id in data_format_dict:
      data_format_dict[animal_id].append(cow_data)
    else:
      data_format_dict[animal_id] = [cow_data]
  return data_format_dict
  

### Function which will check if a particular blob for an animal ID exists or not.

In [0]:
def check_if_blob_exists(container_name, blob_name):
    start_time = datetime.now()
    append_blob_service = AppendBlobService(account_name = storage_account_name, account_key = storage_account_key)

    if append_blob_service.exists(container_name, blob_name):
        print("\nBlob '{0}' found!".format(blob_name));
        end_time = datetime.now()
        print("\n***** Elapsed Time => {0} *****".format(end_time - start_time))
        return True
    else:
        print("\nBlob '{0}' NOT found!".format(blob_name));
        end_time = datetime.now()
        print("\n***** Elapsed Time => {0} *****".format(end_time - start_time))
        return False

### This function will fetch data from API then transform and store it according to animal id in Azure Storage

In [0]:
def data_animal_to_blob(container_name,username,password):
  
    service = AppendBlobService(storage_account_name, storage_account_key, sas_token=None, is_emulated=False, protocol='https',endpoint_suffix='core.windows.net', custom_domain=None, request_session=None, connection_string=None, socket_timeout=None, token_credential=None)
    
    data = csv_to_dict(username,password)
    for animal_id in data:
      for animal_data in data[animal_id]:
        if check_if_blob_exists('animal-data', str(animal_id)):
          
          service.append_blob_from_text(container_name, animal_id, str(animal_data), encoding='utf-8', validate_content=False, maxsize_condition=None, progress_callback=None, lease_id=None, timeout=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None)
          
          print("Data appended successfully to ",animal_id)
          
        else:
          
          service.create_blob(container_name, animal_id, content_settings=None, metadata=None, lease_id=None, if_modified_since=None,if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None)
          
          service.append_blob_from_text(container_name, animal_id, str(animal_data), encoding='utf-8', validate_content=False, maxsize_condition=None, progress_callback=None, lease_id=None, timeout=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None)#           
          print("Data uploaded successfully to ",animal_id)
    
    
    

# Main

### Credentials of Azure Account

In [0]:
username = ""
password = ""
container_name = ''
storage_account_name = ''
storage_account_key = ''
my_connection_string = ''

### Running all the three created functions

In [0]:
# fetch_data_db(username,password)
# restAPI_data_to_dataFrame(username,password)
# csv_to_dict(username,password) 

### Next command will take a lot of time to run

In [0]:
# data_animal_to_blob(container_name,username,password)
# Took 25 minutes in last run